In [1]:
# import
import pandas as pd
import numpy as np
from plotly_charts import interactFigure
from numba import njit
from funcs import vwap, cPIN
from backtester import getLong, getShort
import talib

In [2]:
T = np.load('tick.npz')['arr_0'].view(np.recarray)
len(T)

889360

In [3]:
fp32 = np.float32
default_fee = 0.05 

@njit(nogil=True)
def backtestLimit(PriceA, qA, qB, fee_percent=default_fee) -> list:
    """Vectorized backtester for limit order strategies"""

    buys = [(int(x), fp32(x)) for x in range(0)]
    sells = [(int(x), fp32(x)) for x in range(0)]
    trades = [(int(x), fp32(x), int(x), fp32(x), int(x), fp32(x), fp32(x)) for x in range(0)]

    pos: int = 0

    for i in range(len(PriceA) - 1):
        price = PriceA[i]

        if price > qA[i]:
            delta_pos = -min(pos + 1, 1)
        elif price < qB[i]:
            delta_pos = min(1 - pos, 1)
        else:
            delta_pos = 0

        k = i + 1
        if delta_pos > 0:
            buys.append((k, qB[k]))
        elif delta_pos < 0: 
            sells.append((k, qA[k]))

        if len(sells) > 0 and len(buys) > 0:
            k_buy, buy = buys.pop(0)
            k_sell, sell = sells.pop(0)
            d_rawPnL = sell - buy
            fee = fee_percent / 100 * (sell + buy)
            d_PnL = d_rawPnL - fee
            if delta_pos < 0:
                trades.append((k_buy, buy, k_sell, sell, -delta_pos, d_PnL, fee))
            else:
                trades.append((k_sell, sell, k_buy, buy, -delta_pos, d_PnL, fee))

        pos += delta_pos

    return trades


def npBacktestLimit(PriceA, qA, qB, fee_percent=default_fee) -> np.ndarray:
    """Converts trades from the limit-backtester to structured array"""

    trades = backtestLimit(PriceA, qA, qB, fee_percent=fee_percent)
    TPairTrade = [('X0', int), ('Price0', float), ('X1', int), ('Price1', float),
                  ('Size', float), ('Profit', float), ('Fee', float)]
    return np.array(trades, dtype=TPairTrade).view(np.recarray)

In [4]:
# declare chart linestyles
lines = {'Tick': dict(color='gray', opacity=0.5),
         'Center': dict(color='blue', opacity=0.5),
         'qA': dict(color='red', opacity=0.5, dash='dot'),
         'qB': dict(color='green', opacity=0.5, dash='dot'),
         'Profit': dict(color='black', width=8, opacity=0.1, secondary_y=True, shape='hv'),
         'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
         'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1)),
         'OSC': dict(color='orange', row=2, col=1, opacity=0.6),
         'VPIN': dict(color='cyan', row=3, col=1, opacity=0.6)
        }

In [5]:
# trade model func (return fitness and dataseries to be charted)
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1)):
    center = vwap(T.PriceA, T.VolumeA, Period)
    std = pd.Series(T.PriceA).rolling(Period).std().values
    qA = center + std*StdDev
    qB = center - std*StdDev

    trades = npBacktestLimit(T.PriceA, qA, qB)
    lx, ly = getLong(trades)
    sx, sy = getShort(trades)
    
    r = len(T.DateTimeA) // 1000
    return trades, dict(Tick=T.PriceA[::r], Center=center[::r], qA=qA[::r], qB=qB[::r], 
                        Profit=dict(x=trades.X1//r, y=trades.Profit.cumsum() + trades.Fee.cumsum()),
                        Buy=dict(x=lx//r, y=ly), Sell=dict(x=sx//r, y=sy),
                        OSC=talib.ROCR(T.PriceA, Period)[::r],
                        VPIN=cPIN(T, Period)[::r])

In [6]:
interactFigure(model, lines, rows=3, height=650)